<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/semantic_vector6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Pretrained sentence-transformers models](https://www.sbert.net/docs/pretrained_models.html)

[Our chosen light model in hugging face](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

[ Convert a collection of raw documents to a matrix of TF-IDF features ](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
)

In [ ]:

!pip install -U sentence-transformers


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import pandas as pd
import json

In [ ]:
# Read log templates file into a DataFrame
df = pd.read_csv('/content/HDFS_templates.csv')
df.head(3)

,EventId,EventTemplate
0,E1,<*>Adding an already existing block<*>
1,E2,<*>Verification succeeded for<*>
2,E3,<*>Served block<*>to<*>


In [ ]:
# we keep some stop words such as on, over, not, .. which can have significant meaning
stop_words = {
    'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren',
    "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by',
    'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don',
    "don't", 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't",
    'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how',
    'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me',
    'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'nor', 'now', 'o',
    'of', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'own', 're', 's', 'same', 'shan',
    "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than',
    'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this',
    'those', 'through', 'to', 'too', 'until', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren',
    "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't",
    'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself',
    'yourselves'
}

# Pre-compiling the regular expression pattern using re.compile() can improve the performance of the regular expression operations
pattern = re.compile(r'\W+|\d')

In [ ]:
def tokenized(text):
    """
    Normalize text to extract most salient tokens
    """
    # Replace special characters with space and remove digits
    text = pattern.sub(' ', text)

    # Convert camel case to snake case, then replace _ with space
    text = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
    text = re.sub('([a-z0-9])([A-Z])', r'\1_\2', text).lower().replace('_', ' ')

    normalized_tokens = [w for w in text.split() if w not in stop_words]

    # Return the filtered sentence, our output will be sentences not a list of words
    return ' '.join(normalized_tokens)


In [ ]:
tokenized_template = [tokenized(sentence) for sentence in df['EventTemplate'] ]
print(tokenized_template)

['adding already existing block', 'verification succeeded', 'served block', 'got exception serving', 'receiving block src dest', 'received block src dest size', 'write block received exception', 'packet responder block interrupted', 'received block size', 'packet responder exception', 'packet responder block terminating', 'exception writing block mirror', 'receiving empty packet block', 'exception receive block block', 'changing block file offset block meta file offset', 'transmitted block', 'failed transfer got', 'starting thread transfer block', 'reopen block', 'unexpected error trying delete block block info not found volume map', 'deleting block file', 'block name system allocate block', 'block name system delete added invalid set', 'block removing block needed replications not belong file', 'block ask replicate', 'block name system add stored block block map updated added size', 'block name system add stored block redundant add stored block request received on size', 'block name s

In [ ]:
for i in range(10):
  print('Template ',i,' : ', df['EventTemplate'][i])
  print('Tokenized',i,' : ', tokenized_template[i])
  print('- - - - - - - - - - - - - - - - -')


Template  0  :  <*>Adding an already existing block<*>
Tokenized 0  :  adding already existing block
- - - - - - - - - - - - - - - - -
Template  1  :  <*>Verification succeeded for<*>
Tokenized 1  :  verification succeeded
- - - - - - - - - - - - - - - - -
Template  2  :  <*>Served block<*>to<*>
Tokenized 2  :  served block
- - - - - - - - - - - - - - - - -
Template  3  :  <*>Got exception while serving<*>to<*>
Tokenized 3  :  got exception serving
- - - - - - - - - - - - - - - - -
Template  4  :  <*>Receiving block<*>src:<*>dest:<*>
Tokenized 4  :  receiving block src dest
- - - - - - - - - - - - - - - - -
Template  5  :  <*>Received block<*>src:<*>dest:<*>of size<*>
Tokenized 5  :  received block src dest size
- - - - - - - - - - - - - - - - -
Template  6  :  <*>writeBlock<*>received exception<*>
Tokenized 6  :  write block received exception
- - - - - - - - - - - - - - - - -
Template  7  :  <*>PacketResponder<*>for block<*>Interrupted<*>
Tokenized 7  :  packet responder block interr

# **PART1 : Using light sentence-transformers model**

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
templates2vec1 = [model.encode(sentence) for sentence in tokenized_template]
templates2vec1[0].shape

(384,)

In [50]:
# save templates2vec1 in a json file

# Convert the list of vectors to a list of lists
vector_list = [vector.tolist() for vector in templates2vec1]

# Save the list to a file in JSON format
with open('templates2vec.json', 'w') as file:
    json.dump(vector_list, file)


how to load:


```
# Load the data from the JSON file
with open('templates2vec.json', 'r') as file:
    vector_list = json.load(file)

# Convert the list of lists back to a list of numpy arrays
vectors2 = [vector for vector in vector_list]

# Now 'vectors' contains the original list of vectors loaded from the file
```



In [49]:
similarities = cosine_similarity(templates2vec1[0].reshape(1, -1), templates2vec1[:30]) # similarities.shape = (1, 30)
for i, similarity in enumerate(similarities[0]):
    print("Similarity:", similarity)
    print(tokenized_template[0])
    print(tokenized_template[i])
    print('------------------------------------')


Similarity: 1.0000001
adding already existing block
adding already existing block
------------------------------------
Similarity: 0.021313239
adding already existing block
verification succeeded
------------------------------------
Similarity: 0.309552
adding already existing block
served block
------------------------------------
Similarity: 0.008829565
adding already existing block
got exception serving
------------------------------------
Similarity: 0.3424176
adding already existing block
receiving block src dest
------------------------------------
Similarity: 0.293378
adding already existing block
received block src dest size
------------------------------------
Similarity: 0.389035
adding already existing block
write block received exception
------------------------------------
Similarity: 0.22932869
adding already existing block
packet responder block interrupted
------------------------------------
Similarity: 0.35335255
adding already existing block
received block size
-----

In [ ]:

# Compute pairwise similarities
similarities = cosine_similarity(templates2vec1)
similarities = np.triu(similarities, k=1)  # Exclude diagonal and lower triangular elements, change them into zero

# Find indices where similarity exceeds threshold
indices = np.where(similarities > 0.75)

# Iterate over similar template pairs
for i, j in zip(indices[0], indices[1]):
    similarity = similarities[i, j]
    print('similarity =', similarity)
    print('template', i, ':', tokenized_template[i])
    print('template', j, ':', tokenized_template[j])
    print('--------------------------------')


similarity = 0.78556585
template 4 : receiving block src dest
template 5 : received block src dest size
--------------------------------
similarity = 0.81611705
template 5 : received block src dest size
template 8 : received block size
--------------------------------
similarity = 0.87190074
template 7 : packet responder block interrupted
template 10 : packet responder block terminating
--------------------------------
similarity = 0.7676006
template 25 : block name system add stored block block map updated added size
template 26 : block name system add stored block redundant add stored block request received on size
--------------------------------
similarity = 0.8484187
template 26 : block name system add stored block redundant add stored block request received on size
template 27 : block name system add stored block add stored block request received on size not belong file
--------------------------------


# **PART2 : using word2vec and tff-idf**

In [ ]:
import gensim.downloader

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
# Download the 'word2vec-google-news-300' embeddings
word2vec = gensim.downloader.load('word2vec-google-news-300')

[=================================================-] 100.0% 1662.3/1662.8MB downloaded


In [ ]:
word2vec.most_similar('add')

[('Adding', 0.578197181224823),
 ('Add', 0.5681695938110352),
 ('adds', 0.5657491683959961),
 ('bring', 0.5600515604019165),
 ('augment', 0.5254836678504944),
 ('create', 0.521090567111969),
 ('incorporate', 0.504277765750885),
 ('expand', 0.49926939606666565),
 ('combine', 0.49364253878593445),
 ('elevate', 0.4872509241104126)]

In [ ]:
# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Compute TF-IDF features
tfidf_features = tfidf_vectorizer.fit_transform(tokenized_template)

# Access the TF-IDF feature matrix
print(tfidf_features.toarray().shape)

(30, 66)


In [ ]:
tfidf_vectorizer.vocabulary_

{'adding': 2,
 'already': 4,
 'existing': 15,
 'block': 7,
 'verification': 62,
 'succeeded': 52,
 'served': 45,
 'got': 19,
 'exception': 14,
 'serving': 46,
 'receiving': 36,
 'src': 49,
 'dest': 11,
 'received': 35,
 'size': 48,
 'write': 64,
 'packet': 32,
 'responder': 44,
 'interrupted': 21,
 'terminating': 54,
 'writing': 65,
 'mirror': 25,
 'empty': 12,
 'receive': 34,
 'changing': 8,
 'file': 17,
 'offset': 30,
 'meta': 24,
 'transmitted': 58,
 'failed': 16,
 'transfer': 57,
 'starting': 50,
 'thread': 55,
 'reopen': 39,
 'unexpected': 60,
 'error': 13,
 'trying': 59,
 'delete': 9,
 'info': 20,
 'not': 29,
 'found': 18,
 'volume': 63,
 'map': 23,
 'deleting': 10,
 'name': 27,
 'system': 53,
 'allocate': 3,
 'added': 1,
 'invalid': 22,
 'set': 47,
 'removing': 38,
 'needed': 28,
 'replications': 42,
 'belong': 6,
 'ask': 5,
 'replicate': 40,
 'add': 0,
 'stored': 51,
 'updated': 61,
 'redundant': 37,
 'request': 43,
 'on': 31,
 'pending': 33,
 'replication': 41,
 'monitor': 26,

In [ ]:
i = 10 # which template

print('template',str(i),' : ',tokenized_template[i])
print('*****************************')
for word in tokenized_template[i].split():
  print(word,' : ', tfidf_features.toarray()[i][tfidf_vectorizer.vocabulary_[word]])


template 10  :  packet responder block terminating
*****************************
packet  :  0.49503408329070714
responder  :  0.5341425044741978
block  :  0.19947366861058732
terminating  :  0.6556243564920345


In [ ]:
# Get the vocabulary dictionary and word indices
dic = tfidf_vectorizer.vocabulary_

matrix_weight = tfidf_features.toarray()

embedding_shape = word2vec.get_vector('word').shape
num_templates = len(tokenized_template)
templates2vec2 = np.zeros((num_templates, embedding_shape[0]))

missing_vectors = [] # Collect words without proper vectors
for i, sentence in enumerate(tokenized_template):
    vector = np.zeros(embedding_shape)
    for word in sentence.split():
        j = dic.get(word)  # If the key is not present, dic.get(word)(or dic.get(word, default_value)) will return None (or any default value you provide), while dic[word] will raise a KeyError if the key is not found.
        if j is not None:
            vector += matrix_weight[i, j] * word2vec.get_vector(word)
        else:
            missing_vectors.append(word)
    templates2vec2[i] = vector

In [ ]:
similarities = cosine_similarity(templates2vec2[0].reshape(1, -1), templates2vec2[:30]) # similarities.shape = (1, 30)
for i, similarity in enumerate(similarities[0]):
    print("Similarity:", similarity)
    print(tokenized_template[0])
    print(tokenized_template[i])
    print('------------------------------------')

Similarity: 1.0000000000000004
adding already existing block
adding already existing block
------------------------------------
Similarity: 0.21925066637897228
adding already existing block
verification succeeded
------------------------------------
Similarity: 0.1504715362121909
adding already existing block
served block
------------------------------------
Similarity: 0.24609330762729548
adding already existing block
got exception serving
------------------------------------
Similarity: 0.15985740844471213
adding already existing block
receiving block src dest
------------------------------------
Similarity: 0.1781226566225514
adding already existing block
received block src dest size
------------------------------------
Similarity: 0.24656939608111597
adding already existing block
write block received exception
------------------------------------
Similarity: 0.1065335677729374
adding already existing block
packet responder block interrupted
------------------------------------
Simi

In [ ]:
# Compute pairwise similarities
similarities = cosine_similarity(templates2vec2)
similarities = np.triu(similarities, k=1)  # Exclude diagonal and lower triangular elements, change them into zero

# Find indices where similarity exceeds threshold
indices = np.where(similarities > 0.75)

# Iterate over similar template pairs
for i, j in zip(indices[0], indices[1]):
    similarity = similarities[i, j]
    print('similarity =', similarity)
    print('template', i, ':', tokenized_template[i])
    print('template', j, ':', tokenized_template[j])
    print('--------------------------------')

similarity = 0.8470709133866174
template 4 : receiving block src dest
template 5 : received block src dest size
--------------------------------
similarity = 0.7563605133545468
template 7 : packet responder block interrupted
template 9 : packet responder exception
--------------------------------
similarity = 0.7578512705674404
template 9 : packet responder exception
template 10 : packet responder block terminating
--------------------------------
similarity = 0.7589474554165527
template 21 : block name system allocate block
template 25 : block name system add stored block block map updated added size
--------------------------------
similarity = 0.8244604201949224
template 25 : block name system add stored block block map updated added size
template 26 : block name system add stored block redundant add stored block request received on size
--------------------------------
similarity = 0.830150535591593
template 25 : block name system add stored block block map updated added size
templ

In [ ]:
import random

numbers = list(range(30))  # Create a list of numbers from 0 to 29
pairs = random.sample(numbers, k=20)  # Choose 20 random numbers from the list

# Create pairs of numbers
pairs = [(pairs[i], pairs[i+1]) for i in range(0, len(pairs), 2)]

# Print the pairs
for pair in pairs:
    print(pair)

(11, 22)
(18, 20)
(5, 23)
(2, 6)
(10, 9)
(12, 8)
(13, 17)
(28, 16)
(29, 26)
(19, 25)


In [ ]:
# comparison between two semntic vectors(templates2vec1 vs templates2vec2) resulted from two diffrent models

import random

numbers = list(range(30))  # Create a list of numbers from 0 to 29
pairs = random.sample(numbers, k=20)  # Choose 20 random numbers from the list

# Create pairs of numbers
pairs = [(pairs[i], pairs[i+1]) for i in range(0, len(pairs), 2)]

similarities1 = cosine_similarity(templates2vec1)
similarities2 = cosine_similarity(templates2vec2)
# Print the pairs
for pair in pairs:
  print('Template',str(pair[0]),' : ',tokenized_template[pair[0]])
  print('Template',str(pair[1]),' : ',tokenized_template[pair[1]])
  print('Similarity (Model 1) = ',similarities1[pair])
  print('Similarity (Model 2) = ',similarities2[pair])
  print('****************************************************')
  print()


Template 14  :  changing block file offset block meta file offset
Template 29  :  block ask delete
Similarity (Model 1) =  0.12572029
Similarity (Model 2) =  0.4072718089695322
****************************************************

Template 11  :  exception writing block mirror
Template 27  :  block name system add stored block add stored block request received on size not belong file
Similarity (Model 1) =  0.27382597
Similarity (Model 2) =  0.3370542315265763
****************************************************

Template 4  :  receiving block src dest
Template 23  :  block removing block needed replications not belong file
Similarity (Model 1) =  0.4007976
Similarity (Model 2) =  0.40222270132852656
****************************************************

Template 6  :  write block received exception
Template 20  :  deleting block file
Similarity (Model 1) =  0.44380224
Similarity (Model 2) =  0.32954748458811456
****************************************************

Template 24  :  block